# Vamos a leer el cluster 4 y generar el promedio

vamos por paso




## Preparo todo

In [19]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [20]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend



#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [21]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_4 = pd.read_parquet(DATOS_DIR +'df_cluster_4.parquet')
df_cluster_4.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [22]:
df_cluster_4.head()

,customer_id,product_id,date_mes,first_customer_date,last_customer_date,first_product_date,last_product_date,tn,dif_tn,dif_por,vida_prod,vida_customer,meses_on,meses_off,vida_restante,anio,mes,secuencia,trimestre,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,periodo,stock_final,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect,product_cust_id
0,10001,20010,2017-01,2017-01,2019-12,2017-01,2019-12,1.319,0.0,0.0,4,4,4,0,60,2017,1,1,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Polvo,HC_ROPA LAVADO_Polvo_LIMPIEX,HC_ROPA LAVADO_Polvo_LIMPIEX_400.0,201701,NaN,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47,20010_10001
1,10001,20010,2017-02,2017-01,2019-12,2017-01,2019-12,0.114,0.0,0.0,5,5,5,0,60,2017,2,2,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Polvo,HC_ROPA LAVADO_Polvo_LIMPIEX,HC_ROPA LAVADO_Polvo_LIMPIEX_400.0,201702,NaN,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68,20010_10001
2,10001,20010,2017-03,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,6,6,0,1,60,2017,3,3,1,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Polvo,HC_ROPA LAVADO_Polvo_LIMPIEX,HC_ROPA LAVADO_Polvo_LIMPIEX_400.0,201703,NaN,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95,20010_10001
3,10001,20010,2017-04,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,7,7,0,2,60,2017,4,4,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Polvo,HC_ROPA LAVADO_Polvo_LIMPIEX,HC_ROPA LAVADO_Polvo_LIMPIEX_400.0,201704,NaN,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19,20010_10001
4,10001,20010,2017-05,2017-01,2019-12,2017-01,2019-12,0.000,0.0,0.0,8,8,0,3,60,2017,5,5,2,HC,HC_ROPA LAVADO,HC_ROPA LAVADO_Polvo,HC_ROPA LAVADO_Polvo_LIMPIEX,HC_ROPA LAVADO_Polvo_LIMPIEX_400.0,201705,NaN,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76,20010_10001


## voy a generar simplemente el promedio de los últimos 12 meses por producto

In [23]:
# a partir del df_cluster_4 genero df_resto con solamente la suma por date_mes de tn para cada product_id y product_id
df_resto = df_cluster_4.groupby(['product_id', 'date_mes'])['tn'].sum().reset_index()
df_resto.head()

,product_id,date_mes,tn
0,20001,2017-01,23.147
1,20001,2017-02,15.670
2,20001,2017-03,12.357
3,20001,2017-04,8.887
4,20001,2017-05,27.255


In [24]:
df_resto.dtypes

,0
product_id,int64
date_mes,period[M]
tn,float64


In [25]:
df_2018 = df_resto[(df_resto['date_mes'] >= '2018-01') & (df_resto['date_mes'] <= '2018-12')]

In [26]:
# 3. Agrupamos por product_id y calculamos el promedio de tn en ese período
df_promedio = (
    df_2018.groupby('product_id', as_index=False)['tn']
    .mean()
    .rename(columns={'tn': 'tn_prom_2018'})
)

# 4. El resultado es un DataFrame con columnas: product_id | tn_prom_2018
print(df_promedio.head())

   product_id  tn_prom_2018
0       20001        23.111
1       20002        14.977
2       20003        13.621
3       20004        10.495
4       20005         5.497


In [27]:
productos_validar.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [28]:
productos_validar.shape

(780, 1)

In [29]:
df_promedio.shape

(1051, 2)

In [30]:
# Merge para obtener el promedio solo para los product_id que están en productos_validar
df_validar_promedio = productos_validar.merge(
    df_promedio,
    on='product_id',
    how='left'
)

# Ahora df_validar_promedio tendrá dos columnas: product_id y tn_prom_2018
print(df_validar_promedio.head())

   product_id  tn_prom_2018
0       20001        23.111
1       20002        14.977
2       20003        13.621
3       20004        10.495
4       20005         5.497


In [ ]:
df_validar_promedio.shape

(780, 2)

In [31]:
df_validar_promedio.to_csv(DATOS_DIR +'df_cluster4_promedio.csv', index=False)